In [4]:
import pandas as pd
import numpy as np
import bs4
import requests

In [5]:
cpu_benchmarks = "https://browser.geekbench.com/processor-benchmarks"
gpu_benchmarks = "https://browser.geekbench.com/vulkan-benchmarks"

# CPU

In [9]:
laptops = pd.read_csv("../data/final_data.csv", index_col=0)
laptops.head()

brand                                            img_url   
0      HP  https://encrypted-tbn0.gstatic.com/images?q=tb...  \
1  Lenovo  https://encrypted-tbn0.gstatic.com/images?q=tb...   
2  Huawei  https://encrypted-tbn0.gstatic.com/images?q=tb...   
3    Dell  https://encrypted-tbn0.gstatic.com/images?q=tb...   
4    Asus  https://encrypted-tbn0.gstatic.com/images?q=tb...   

            laptop_name  display_size       processor_type   
0  Notebook 14-df0008nx          14.0  Intel Celeron N4000  \
1    IdeaPad 330S-14IKB          14.0  Intel Core i5-8250U   
2      MateBook D Volta          14.0  Intel Core i5-8250U   
3      Inspiron 15 3567          15.6  Intel Core i3-7020U   
4    VivoBook 15 X510UR          15.6  Intel Core i7-8550U   

                 graphics_card  disk_space  discount_price  ratings_5max   
0        Intel HD Graphics 600        64.0          1259.0           0.0  \
1       Intel UHD Graphics 620      1024.0          1849.0           3.3   
2  NVIDIA GeForce MX150 (2 GB)       256.0          2999.0           0.0   
3        Intel HD Graphics 620      1024.0          1849.0           0.0   
4  NVIDIA GeForce 930MX (2 GB)      1024.0          2499.0           0.0   

  disk_type  
0      eMMC  
1       HDD  
2       SSD  
3       HDD  
4       HDD

In [10]:
!ls ..

api  data  notebooks  README.md


In [11]:
response = requests.get(cpu_benchmarks)
response.status_code

200

In [17]:
soup = bs4.BeautifulSoup(response.text)

In [18]:
single_core = soup.select(".table")[0]

In [19]:
columns = ["Processor", "Score"]
single_core.select("thead th")

[<th class="name">Processor</th>,
 <th class="score">Score</th>,
 <th class="graph"></th>]

In [20]:
processors = single_core.select("tbody tr")
def parse_processor(processor):
    
    name = processor.select("a")[0].text.strip()
    
    description = processor.select(".description")[0].text.strip().split()
    clock_speed = description[0].strip()
    cores = description[2][1:].strip()
    
    score = processor.select(".score")[0].text.strip()
    
    data = {
        "processor_type": name,
        "cpu_clock_speed": float(clock_speed),
        "cpu_cores": int(cores),
        "cpu_score": int(score)
    }
    return data

processors = pd.DataFrame(map(parse_processor, processors))
processors.head()

processor_type  cpu_clock_speed  cpu_cores  cpu_score
0  Intel Core i9-13900KS              3.2         24       3072
1  Intel Core i9-13900KF              3.0         24       2949
2   Intel Core i9-13900K              3.0         24       2940
3      AMD Ryzen 9 7950X              4.5         16       2874
4    AMD Ryzen 9 7950X3D              4.2         16       2873

In [21]:
processors.dtypes

processor_type      object
cpu_clock_speed    float64
cpu_cores            int64
cpu_score            int64
dtype: object

# GPU

In [22]:
response = requests.get(gpu_benchmarks)
response.status_code

200

In [24]:
soup = bs4.BeautifulSoup(response.text)

In [25]:
table = soup.select("table")[0]
cols = ["Device", "Score"]
table.select("thead tr")

[<tr class="stacked-heading">
 <th class="name">Device</th>
 <th class="score">Score</th>
 <th class="graph"></th>
 </tr>]

In [26]:
table = soup.select("tbody tr")
def parse_gpu(row):
    data = {
        "graphics_card": row.select_one(".name").text.strip(),
        "gpu_score": int(row.select_one(".score").text.strip())
    }
    return data

gpu = pd.DataFrame(map(parse_gpu, table))
gpu.head()

graphics_card  gpu_score
0                NVIDIA GeForce RTX 4090     254154
1  AMD Radeon RX 7900 XTX (RADV GFX1100)     224764
2         NVIDIA RTX 6000 Ada Generation     222276
3                 AMD Radeon RX 7900 XTX     211013
4                NVIDIA GeForce RTX 4080     197741

In [27]:
gpu.dtypes

graphics_card    object
gpu_score         int64
dtype: object

In [28]:
processors

processor_type  cpu_clock_speed  cpu_cores  cpu_score
0    Intel Core i9-13900KS              3.2         24       3072
1    Intel Core i9-13900KF              3.0         24       2949
2     Intel Core i9-13900K              3.0         24       2940
3        AMD Ryzen 9 7950X              4.5         16       2874
4      AMD Ryzen 9 7950X3D              4.2         16       2873
..                     ...              ...        ...        ...
794            AMD A4-5000              1.5          4        161
795    Intel Celeron N3050              1.6          2        154
796    Intel Atom x5-Z8350              1.4          4        134
797    Intel Atom x5-Z8300              1.4          4        131
798               AMD C-60              1.0          2         76

[799 rows x 4 columns]

In [65]:
laptops.dropna(inplace=True)

In [66]:
cols = laptops.dtypes[laptops.dtypes == "object"].index
for col in cols:
    laptops[col] = laptops[col].apply(lambda x: x.strip())
laptops.head()

brand                                            img_url  \
0      HP  https://encrypted-tbn0.gstatic.com/images?q=tb...   
1  Lenovo  https://encrypted-tbn0.gstatic.com/images?q=tb...   
2  Huawei  https://encrypted-tbn0.gstatic.com/images?q=tb...   
3    Dell  https://encrypted-tbn0.gstatic.com/images?q=tb...   
4    Asus  https://encrypted-tbn0.gstatic.com/images?q=tb...   

            laptop_name  display_size       processor_type  \
0  Notebook 14-df0008nx          14.0  Intel Celeron N4000   
1    IdeaPad 330S-14IKB          14.0  Intel Core i5-8250U   
2      MateBook D Volta          14.0  Intel Core i5-8250U   
3      Inspiron 15 3567          15.6  Intel Core i3-7020U   
4    VivoBook 15 X510UR          15.6  Intel Core i7-8550U   

                 graphics_card    disk_space  discount_price  old_price  \
0        Intel HD Graphics 600  64 GB (eMMC)          1259.0     1259.0   
1       Intel UHD Graphics 620      1 TB HDD          1849.0     2099.0   
2  NVIDIA GeForce MX150 (2 GB)    256 GB SSD          2999.0     3799.0   
3        Intel HD Graphics 620      1 TB HDD          1849.0     1849.0   
4  NVIDIA GeForce 930MX (2 GB)      1 TB HDD          2499.0     3149.0   

   ratings_5max  
0           0.0  
1           3.3  
2           0.0  
3           0.0  
4           0.0

In [67]:
laptops.join(processors.set_index("processor_type"), on="processor_type")

brand                                            img_url  \
0        HP  https://encrypted-tbn0.gstatic.com/images?q=tb...   
1    Lenovo  https://encrypted-tbn0.gstatic.com/images?q=tb...   
2    Huawei  https://encrypted-tbn0.gstatic.com/images?q=tb...   
3      Dell  https://encrypted-tbn0.gstatic.com/images?q=tb...   
4      Asus  https://encrypted-tbn0.gstatic.com/images?q=tb...   
..      ...                                                ...   
199    Dell  https://encrypted-tbn0.gstatic.com/images?q=tb...   
200  Lenovo  https://encrypted-tbn0.gstatic.com/images?q=tb...   
201  Huawei  https://encrypted-tbn0.gstatic.com/images?q=tb...   
202   Apple  https://encrypted-tbn0.gstatic.com/images?q=tb...   
203   Apple  https://encrypted-tbn0.gstatic.com/images?q=tb...   

                          laptop_name  display_size        processor_type  \
0                Notebook 14-df0008nx          14.0   Intel Celeron N4000   
1                  IdeaPad 330S-14IKB          14.0   Intel Core i5-8250U   
2                    MateBook D Volta          14.0   Intel Core i5-8250U   
3                    Inspiron 15 3567          15.6   Intel Core i3-7020U   
4                  VivoBook 15 X510UR          15.6   Intel Core i7-8550U   
..                                ...           ...                   ...   
199                  Inspiron 15 3567          15.6   Intel Core i5-7200U   
200               IdeaPad 320-15IKBRN          15.6   Intel Core i5-8250U   
201                        MateBook D          15.6   Intel Core i7-8550U   
202  MacBook Pro (Retina + Touch Bar)          15.4  Intel Core i7 6 Core   
203  MacBook Pro (Retina + Touch Bar)          15.4  Intel Core i7 6 Core   

                    graphics_card           disk_space  discount_price  \
0           Intel HD Graphics 600         64 GB (eMMC)          1259.0   
1          Intel UHD Graphics 620             1 TB HDD          1849.0   
2     NVIDIA GeForce MX150 (2 GB)           256 GB SSD          2999.0   
3           Intel HD Graphics 620             1 TB HDD          1849.0   
4     NVIDIA GeForce 930MX (2 GB)             1 TB HDD          2499.0   
..                            ...                  ...             ...   
199     AMD Radeon R5-M430 (2 GB)             1 TB HDD          2249.0   
200                  Intel GMA HD             1 TB HDD          2099.0   
201   NVIDIA GeForce MX150 (2 GB)  128 GB SSD/1 TB HDD          3299.0   
202  Radeon Pro 555X GDDR5 (4 GB)           256 GB SSD         10199.0   
203  Radeon Pro 560X GDDR5 (4 GB)           512 GB SSD         11899.0   

     old_price  ratings_5max  cpu_clock_speed  cpu_cores  cpu_score  
0       1259.0           0.0              1.1        2.0      329.0  
1       2099.0           3.3              NaN        NaN        NaN  
2       3799.0           0.0              NaN        NaN        NaN  
3       1849.0           0.0              2.3        2.0      726.0  
4       3149.0           0.0              NaN        NaN        NaN  
..         ...           ...              ...        ...        ...  
199     2249.0           3.3              NaN        NaN        NaN  
200     2099.0           3.8              NaN        NaN        NaN  
201     3299.0           4.0              NaN        NaN        NaN  
202    10199.0           0.0              NaN        NaN        NaN  
203    11899.0           4.4              NaN        NaN        NaN  

[203 rows x 13 columns]

In [37]:
processors[["processor_type", "cpu_score"]].set_index("processor_type").to_clipboard()

In [36]:
processors.shape

(799, 4)

In [41]:
list(laptops.processor_type.unique())

['Intel Celeron N4000',
 'Intel Core i5-8250U',
 'Intel Core i3-7020U',
 'Intel Core i7-8550U',
 'Intel Core i5 Dual Core',
 'AMD A9-9425',
 'Intel Core i7-8565U',
 'Intel Core i7 6 Core',
 'Intel Core i5 Quad Core',
 'Intel Core i5-8265U',
 'Intel Core M3',
 'Intel Core i7-8750H',
 'Intel Core i3-8145U',
 'Intel Celeron 4205U',
 'Intel Celeron N3060',
 'Intel Celeron N3350',
 'Intel Core i5-7200U',
 'Intel Core i9',
 'Intel Core i7-7820HK',
 'Intel Core i7-7700HQ',
 'Intel Core i7-6700HQ',
 'Intel Core i7 Quad Core',
 'Intel Core i7-7500U',
 'Intel Core i3-5005U',
 'Intel Core i3-6006U',
 'Intel Core i3-8130U']